In [1]:
import pandas as pd
import numpy as np
import sklearn as skl

In [2]:
candles = pd.read_pickle('CAD_USD_H2')

In [11]:
candles

,rsi_14,MACD_12_26_9,stochastic_k,stochastic_d,spread,mid_c_prev,long,short,rsi_14_lag_1,MACD_12_26_9_lag_1,...,stochastic_range_k_lag_6,stochastic_range_d_lag_6,stochastic_range_k_lag_7,stochastic_range_d_lag_7,stochastic_range_k_lag_8,stochastic_range_d_lag_8,stochastic_range_k_lag_9,stochastic_range_d_lag_9,stochastic_range_k_lag_10,stochastic_range_d_lag_10
27,81.698530,0.000000,84.558824,94.208642,0.00050,0.97472,False,True,81.698530,0.000000,...,0.230000,0.256248,0.230000,0.256248,0.230000,0.256248,0.230000,0.256248,0.230000,0.256248
28,90.432358,0.000120,95.734597,93.186042,0.00029,0.97646,True,False,90.432358,0.000120,...,0.404000,0.393245,0.404000,0.393245,0.404000,0.393245,0.404000,0.393245,0.404000,0.393245
29,90.910130,0.000196,93.750000,91.347807,0.00029,0.97662,False,True,90.910130,0.000196,...,0.420000,0.409238,0.420000,0.409238,0.420000,0.409238,0.420000,0.409238,0.420000,0.409238
30,93.481013,0.000311,98.032200,95.838933,0.00029,0.97790,True,False,93.481013,0.000311,...,0.548000,0.535740,0.548000,0.535740,0.548000,0.535740,0.548000,0.535740,0.548000,0.535740
31,94.958434,0.000445,96.576320,96.119507,0.00024,0.97919,True,False,94.958434,0.000445,...,0.677000,0.673798,0.677000,0.673798,0.677000,0.673798,0.677000,0.673798,0.677000,0.673798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34994,29.131147,-0.000312,9.685230,8.038271,0.00018,1.27094,False,True,29.131147,-0.000312,...,0.080000,0.066396,0.080000,0.066396,0.080000,0.066396,0.080000,0.066396,0.080000,0.066396
34995,27.836030,-0.000324,2.069858,4.655826,0.00019,1.27026,False,True,27.836030,-0.000324,...,0.017180,0.038643,0.017180,0.038643,0.017180,0.038643,0.017180,0.038643,0.017180,0.038643
34996,28.390643,-0.000289,17.506631,9.753906,0.00020,1.27036,False,True,28.390643,-0.000289,...,0.153883,0.085737,0.153883,0.085737,0.153883,0.085737,0.153883,0.085737,0.153883,0.085737
34997,26.202262,-0.000301,5.426357,8.334282,0.00020,1.26926,False,True,26.202262,-0.000301,...,0.042000,0.064507,0.042000,0.064507,0.042000,0.064507,0.042000,0.064507,0.042000,0.064507


In [3]:
for_short = [col for col in candles.columns if col != 'long']
for_long = [col for col in candles.columns if col != 'short']
candle_features = candles[[col for col in candles.columns if col not in ['long', 'short']]]

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
scaler = StandardScaler()
scaler.fit(candle_features)

scaled_candles = scaler.transform(candle_features)

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [23]:
params = {
    'n_neighbors': list(range(15,26)),
    'weights': ('uniform', 'distance')
}

long_model = GridSearchCV(KNeighborsClassifier(), param_grid=params, cv=5)
short_model = GridSearchCV(KNeighborsClassifier(), param_grid=params, cv=5)

In [32]:
long = candles['long']
short = candles['short']

In [42]:
x_train_candles = scaled_candles[:-3069]
y_train_long = long[:-3069]
y_train_short = short[:-3069]

x_test_candles = scaled_candles[-3069:]
y_test_long = long[-3069:]
y_test_short = short[-3069:]

y_test_long.reset_index(drop=True, inplace=True)
y_test_short.reset_index(drop=True, inplace=True)

In [34]:
long_model.fit(x_train_candles, y_train_long)
short_model.fit(x_train_candles, y_train_short)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
                                         25],
                         'weights': ('uniform', 'distance')})

In [36]:
short_results = pd.DataFrame(short_model.cv_results_)
short_results[['param_n_neighbors', 'param_weights', 'mean_test_score']].sort_values('mean_test_score', ascending=False)

,param_n_neighbors,param_weights,mean_test_score
19,24,distance,0.885195
15,22,distance,0.885195
17,23,distance,0.884911
7,18,distance,0.884848
16,23,uniform,0.884785
12,21,uniform,0.884722
4,17,uniform,0.884722
11,20,distance,0.884659
13,21,distance,0.884375
5,17,distance,0.884280


In [37]:
long_results = pd.DataFrame(long_model.cv_results_)
long_results[['param_n_neighbors', 'param_weights', 'mean_test_score']].sort_values('mean_test_score', ascending=False)

,param_n_neighbors,param_weights,mean_test_score
12,21,uniform,0.896644
13,21,distance,0.896518
16,23,uniform,0.895698
17,23,distance,0.895540
8,19,uniform,0.895509
21,25,distance,0.895383
20,25,uniform,0.895288
11,20,distance,0.895099
19,24,distance,0.895067
9,19,distance,0.895067


In [38]:
final_short_model = KNeighborsClassifier(n_neighbors=24, weights='distance')
final_long_model = KNeighborsClassifier(n_neighbors=21, weights='uniform')

final_short_model.fit(x_train_candles, y_train_long)
final_long_model.fit(x_train_candles, y_train_long)

KNeighborsClassifier(n_neighbors=21)

In [46]:
y_test_long.reset_index(drop=True, inplace=True)
y_test_short.reset_index(drop=True, inplace=True)

In [47]:
short_test_results = pd.DataFrame(final_short_model.predict(x_test_candles), columns=['predictions'])
short_test_results['actual'] = y_test_short
short_takes = short_test_results.loc[short_test_results['predictions'] == True]

long_test_results = pd.DataFrame(final_long_model.predict(x_test_candles), columns=['predictions'])
long_test_results['actual'] = y_test_long
long_takes = long_test_results.loc[long_test_results['predictions'] == True]

In [49]:
print(f"Short Trades:\nTrue accuracy: {short_takes['actual'].mean()} \n Count: {short_takes['actual'].count()}")

Short Trades:
True accuracy: 0.30420499342969776 
 Count: 1522


In [48]:
print(f"Long Trades:\nTrue accuracy: {long_takes['actual'].mean()} \n Count: {long_takes['actual'].count()}")

Long Trades:
True accuracy: 0.9680573663624511 
 Count: 1534


In [60]:
long_wins = long_takes.loc[long_takes['actual'] == True, 'actual'].count()
long_ror = (1.01 ** long_wins) * (0.97 ** (1534-long_wins))
long_ror

587563.8207732333

In [55]:
short_wins = short_takes.loc[short_takes['actual'] == True, 'actual'].count()
short_ror = (1.01 ** short_wins) * (0.97 ** (1522-short_wins))
short_ror

9.818895756608528e-13